# 🧼 PHASE 2 — Data Cleaning Strategy

In [128]:
# Reload Raw Data
import pandas as pd
import numpy as np

df = pd.read_csv("../data/raw/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [129]:
# Fix Incorrect Data Types
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df['TotalCharges'].isnull().sum()

np.int64(11)

In [130]:
# Handle Missing Values
df.loc[df['tenure'] == 0, 'TotalCharges'] = 0
df.isnull().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [131]:
# Encode Target Variable
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})

In [132]:
# Normalize Binary Categorical Columns
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']

for col in binary_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0})

In [133]:
df.drop(columns=["customerID"], inplace=True)

categorical_cols = df.select_dtypes(include="object").columns.tolist()
categorical_cols


C:\Users\Queen\AppData\Local\Temp\ipykernel_23176\699295381.py:3: Pandas4Warning: For backward compatibility, 'str' dtypes are included by select_dtypes when 'object' dtype is specified. This behavior is deprecated and will be removed in a future version. Explicitly pass 'str' to `include` to select them, or to `exclude` to remove them and silence this warning.
See https://pandas.pydata.org/docs/user_guide/migration-3-strings.html#string-migration-select-dtypes for details on how to write code that works with pandas 2 and 3.
  categorical_cols = df.select_dtypes(include="object").columns.tolist()


['gender',
 'MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaymentMethod']

In [134]:
# # One-Hot Encode Multi-Class Categoricals

# df = pd.get_dummies(
#     df,
#     columns=['Contract', 'PaymentMethod', 'InternetService', 'MultipleLines'],
#     drop_first=True
# )

# categorical_cols = ['Contract', 'PaymentMethod', 'InternetService', 'MultipleLines']
# existing_cols = [col for col in categorical_cols if col in df.columns]

# df = pd.get_dummies(df, columns=existing_cols, drop_first=True)

df = pd.get_dummies(
    df,
    columns=categorical_cols,
    drop_first=True
)


In [135]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

num_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']

scaler = StandardScaler()
df[num_cols] = scaler.fit_transform(df[num_cols])

In [136]:
# Final Dataset Check
df.head()
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   SeniorCitizen                          7043 non-null   int64  
 1   Partner                                7043 non-null   int64  
 2   Dependents                             7043 non-null   int64  
 3   tenure                                 7043 non-null   float64
 4   PhoneService                           7043 non-null   int64  
 5   PaperlessBilling                       7043 non-null   int64  
 6   MonthlyCharges                         7043 non-null   float64
 7   TotalCharges                           7043 non-null   float64
 8   Churn                                  7043 non-null   int64  
 9   gender_Male                            7043 non-null   bool   
 10  MultipleLines_No phone service         7043 non-null   bool   
 11  MultipleLines_Y

In [137]:
# Save Clean Dataset
df.to_csv("../data/processed/clean_telco_churn.csv", index=False)

# 🧠 PHASE 3 — Feature Engineering

In [138]:
# Tenure Grouping
df['tenure_group'] = pd.cut(
    df['tenure'],
    bins=[-1, 0, 12, 24, 48, 72],
    labels=['0', '1-12', '13-24', '25-48', '49+']
)

df = pd.get_dummies(df, columns=['tenure_group'], drop_first=True)

In [139]:
# Average Monthly Cost
df['avg_monthly_cost'] = df['TotalCharges'] / (df['tenure'] + 1)

In [140]:
# Contract Risk Indicator
# df['is_month_to_month'] = df['Contract_Month-to-month']
df['is_month_to_month'] = 1 - df['Contract_One year'] - df['Contract_Two year']

In [141]:
# High Charges Flag
high_charge_threshold = df['MonthlyCharges'].quantile(0.75)
df['high_monthly_charge'] = (df['MonthlyCharges'] > high_charge_threshold).astype(int)

In [142]:
# Customer Value Score
df['customer_value'] = df['tenure'] * df['MonthlyCharges']

In [ ]:
# Service Count
service_cols = ['OnlineSecurity', 'OnlineBackup', 
                'DeviceProtection', 'TechSupport', 
                'StreamingTV', 'StreamingMovies']

for col in service_cols:
    df[col] = df[col].map({'Yes': 1, 'No': 0, 'No internet service': 0})

df['num_services'] = df[service_cols].sum(axis=1)

KeyError: 'OnlineSecurity'

: 

In [ ]:
# Auto Payment Flag
df['auto_payment'] = 1 - df['PaymentMethod_Electronic check'] - df['PaymentMethod_Mailed check']

In [ ]:
# Interaction Feature
df['high_cost_monthly_contract'] = (
    df['high_monthly_charge'] * df['is_month_to_month']
)

In [ ]:
# Verify New Features
df.head()
df.shape

(7043, 36)